# 102 LangGraph: Core Concepts

**Workshop**: LangGraph 101 - Express Format  
**Duration**: ~25 minutes  
**Difficulty**: Beginner

📌 **Demo Focus**: Instructor demonstrates core concepts. ~30% content reduction for workshop pacing.

## Learning Objectives

By completing this notebook, you will:
- Understand State and how it flows through graphs
- Learn to create Nodes that process and update state
- Master Graph construction with StateGraph
- Use Edges to connect nodes into workflows
- Build a complete SCM automation workflow

## Prerequisites

- **Knowledge**: Completed 101 (TypedDict basics)

## Table of Contents

1. [Introduction](#1-introduction)
2. [State](#2-state)
3. [Nodes](#3-nodes)
4. [Graphs and Edges](#4-graphs-and-edges)
5. [Summary](#5-summary)

## ✏️ Homework

After workshop, review full version for:
- Advanced state patterns (Annotated types, reducers)
- Progressive complexity examples (3 levels)
- Detailed production SCM patterns
- Extended commit workflow examples
- Conditional edges and tools (preview)

## 1. Introduction

LangGraph is a framework for building stateful, multi-actor applications with Large Language Models (LLMs). At its core, LangGraph provides a graph-based approach to orchestrating workflows where data flows through nodes connected by edges. Think of it like a network diagram - but instead of routers and switches, you have processing nodes that transform state as it moves through your application.

In this notebook, we'll explore the fundamental building blocks of LangGraph without worrying about LLMs or APIs. We'll focus on understanding the core concepts using simple network automation examples.

### Why LangGraph Matters for Network Automation

Without a structured workflow framework:
- Complex automation logic becomes spaghetti code with nested if/else statements
- State management is error-prone and hard to debug
- Adding new steps or changing workflow order requires significant refactoring

With LangGraph:
- Workflows are visual and easy to understand (nodes and edges)
- State is explicitly defined and flows predictably through the graph
- Adding, removing, or reordering steps is as simple as modifying the graph structure

### What We'll Build

In this notebook, we'll learn:
1. How to define **State** using TypedDict (building on 101 concepts)
2. How to create **Nodes** that process state
3. How to construct a **Graph** and connect nodes with **Edges**
4. How to add **Conditional Edges** for dynamic routing
5. How to integrate **Tools** for external capabilities
6. How all these pieces come together in a complete workflow

Let's get started!

### 1.1 Import Core LangGraph Components

Let's import the essential LangGraph components we'll be working with, including visualization tools:

In [ ]:
# Core typing imports
from typing import TypedDict, Annotated, Literal
from pprint import pprint

# LangGraph core components
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode

# Tool decorator
from langchain_core.tools import tool

# Visualization imports
from IPython.display import Image, display

print("✅ Imports successful")
print("\nCore components loaded:")
print("  - TypedDict: For defining state schemas")
print("  - StateGraph: For building graphs")
print("  - START/END: Special graph entry/exit points")
print("  - ToolNode: For integrating external tools")
print("  - tool: Decorator for creating tools")
print("  - IPython.display: For visualizing graphs")

---

## 2. State - The Foundation

### What is State?

**State** is a shared data structure that holds the current information or context of your entire LangGraph application. In simpler terms, it's like your application's memory where it keeps track of variables and data that nodes can access and modify as they execute.

Think of state like the information that needs to be tracked during **SCM address object creation**. When a network administrator creates address objects in Strata Cloud Manager, they need to track multiple pieces of information throughout the process:
- The address object name
- IP address or FQDN value
- Which folder to deploy to
- Description and tags
- Validation status
- Whether the object was created successfully
- Any errors encountered
- Current step in the workflow

This information needs to persist and be accessible throughout the entire configuration workflow. Each task in the workflow (parsing input, validating data, creating the object in SCM) can read this information and update it. That's exactly what **State** does in LangGraph - it's the shared data that flows through your workflow, being read and updated by each step (node).

### Key Points About State

- **Shared Data Structure**: All nodes can access and modify the same state
- **Application Memory**: Keeps track of variables and data throughout execution
- **Type-Safe**: Using TypedDict ensures you know exactly what data exists
- **Centralized**: All information your workflow needs is stored in one place
- **Flows Through Workflow**: Each step receives state, processes it, and returns updates

### Why State Matters

State is the backbone of your LangGraph application. Without properly defined state:
- Each step doesn't know what information is available from previous steps
- Type errors happen at runtime instead of development time
- Debugging becomes difficult because data flow is unclear

With well-defined state:
- Every step knows exactly what data is available
- IDEs provide autocomplete for state fields
- Data flow is explicit and easy to trace
- You can see the complete status of your workflow at any point

### Progressive Complexity in State Design

As you build more sophisticated workflows, your state schemas will evolve. Here's how state complexity typically progresses:

| Level | Complexity | Fields | Example Use Case |
|-------|-----------|--------|------------------|
| **Level 1** | Simple (2-3 fields) | `name`, `ip_netmask`, `folder` | Basic address object creation |
| **Level 2** | Moderate (4-6 fields) | Add: `validated`, `error_message`, `created` | With validation and tracking |
| **Level 3** | Complex (7-10 fields) | Add: `description`, `tags`, `object_id`, `current_step` | Full lifecycle tracking |
| **Level 4** | Advanced (10+ fields) | Add: `api_response`, `errors` (list), `metadata`, `timestamps` | Production-ready with audit trail |

**Recommendation**: Start at Level 1-2 for learning, move to Level 3-4 for production workflows.

### 2.1 Defining State with TypedDict

### 2.3 How State Gets Updated Throughout the Workflow

In LangGraph, each step (node) in your workflow receives the current state, processes it, and returns a dictionary with the fields it wants to update. LangGraph then merges these updates into the state. This is how information flows through your upgrade workflow - each task updates only the parts relevant to it.

---

## 3. Nodes - Processing Units

### What is a Node?

**Nodes** are individual functions or operations that perform specific tasks within your LangGraph workflow. Each node receives an input (typically the current state), processes it, and produces an output (an updated state).

Think of nodes like **stages in an SCM configuration workflow**. When you create address objects in Strata Cloud Manager, you follow a series of steps:
1. **Parse Station**: Extracts address object details from user input
2. **Validation Station**: Validates IP formats, required fields, and folder existence
3. **Preparation Station**: Formats data according to SCM API requirements
4. **Creation Station**: Makes the API call to create the object in SCM
5. **Verification Station**: Confirms the object was created and retrieves its ID

Each station does one specific job. The configuration data moves through each station, and at each stage something specific happens to it. That's exactly what nodes are - each node performs one specific task in your workflow.

### Key Points About Nodes

- **Individual Functions**: Each node is a Python function with a single responsibility
- **Receives State**: Nodes get the current state as input
- **Processes Data**: Performs a specific operation (validation, API call, decision logic)
- **Returns Updates**: Returns a dictionary with state updates
- **Sequential or Parallel**: Nodes can run one after another or in parallel

### Why Nodes Matter

Nodes are where the actual work happens in your LangGraph application:
- Each node has a clear, single responsibility (like validating input or calling the SCM API)
- Breaking workflows into nodes makes them easier to test and debug
- You can reuse nodes across different workflows
- Nodes can be swapped, added, or removed without affecting others

### 3.1 Creating a Simple Node Function

### 3.2 Key Insight: Nodes Are Just Functions

One of the great benefits of nodes is that they're just Python functions:
- You can test them independently before connecting them in a graph
- They receive state as input and return updates as a dictionary
- Each node focuses on one specific responsibility

💡 In the next section, we'll see how to connect multiple nodes using StateGraph and Edges!

---

## 4. Graph and Edges - Connecting the Flow

### What is StateGraph?

Before we discuss edges, let's understand **StateGraph** itself - this is one of the first and most important elements you'll interact with when building LangGraph applications.

**StateGraph** is the builder and compiler for your graph structure. Its main purpose is to:
- Build and compile the graph structure
- Manage nodes, edges, and overall state
- Ensure the workflow operates in a unified way
- Make sure data flows correctly between components

Think of StateGraph like a **configuration management playbook template**. Just as a well-designed playbook template outlines the procedure, defines all the steps, and shows how they connect together, StateGraph defines the structure and flow of your workflow. Before you can execute configuration changes in SCM, you need the playbook. Before you can run a workflow, you need StateGraph.

In practical terms:
- **Playbook template** = StateGraph (defines structure)
- **Individual tasks** = Nodes (steps in the workflow)
- **Task dependencies** = Edges (connections between steps)
- **Executing the playbook** = Running the compiled graph

StateGraph is what ties everything together - without it, you'd just have disconnected nodes and no way to orchestrate them.

### What are Edges?

Now that we understand StateGraph, let's talk about how nodes connect together. That's where **edges** come in.

**Edges** are connections between nodes that determine the flow of execution. They tell your application which node should be executed next after the current one completes its task.

Think of edges like **API call sequences in configuration workflows**. Imagine your SCM configuration workflow as a series of API calls that must happen in order - you can't create an address object before validating the input, and you can't verify creation before making the create call. Each step depends on the previous one completing successfully.

In LangGraph:
- The **API operations** = Nodes (validate, prepare, create, verify)
- The **call sequence** = Edges (connections defining order)
- The **configuration data** = State (flowing through the workflow)

Edges ensure your state flows from one processing step to the next in the correct order.

### Key Points About Edges

- **Connections**: Link nodes together to define workflow sequence
- **Directional**: Flow goes from one specific node to another
- **Determine Flow**: Control which node executes next
- **Two Types**: Simple edges (always follow the same path) and conditional edges (dynamic routing)

### Why Edges Matter

Without edges:
- Nodes would be disconnected and isolated
- You'd have to manually orchestrate the execution order
- No way to define the workflow sequence

With edges:
- Workflow sequence is explicit and visual
- State automatically flows from node to node
- Easy to modify the workflow by changing edges

### 4.1 Building Your First Graph with StateGraph

Now let's put it all together! We'll use **StateGraph** to create a graph, add our nodes, and connect them with edges.

### 4.2 Understanding START and END

Before we connect nodes with edges, let's understand two special elements: **START** and **END**.

**START** is a virtual entry point in LangGraph that marks where the workflow begins. It's important to note that START doesn't perform any operations itself - it simply serves as the designated starting position for the graph's execution. Think of it like the **beginning of your configuration playbook** - it's where you open the document and start following the procedure.

**END** signifies the conclusion of the workflow in LangGraph. When the application reaches this point, the graph's execution completely stops, indicating that all intended processes have been completed. This is like reaching the **final step in your playbook** - the configuration change is complete and there's nothing more to do.

In SCM automation terms:
- **START** = The moment you begin executing your configuration workflow
- **END** = The completion of all configuration tasks and validations

### 4.3 Adding Edges to Define Workflow Flow

Now we'll connect the nodes with edges, using START as our entry point and END as our exit point:

### 4.4 Executing the Runnable

Now let's use our compiled Runnable to execute the SCM address creation workflow!

### 4.4 Practical Example: SCM Address Object Workflow

Now let's build a simpler, focused example using **SCM address objects**. This demonstrates the classic 3-node pattern: **parse → validate → format** that's common in configuration workflows.

This example shows how to transform user input into API-ready SCM configurations!

In [ ]:
# Step 1: Define State Schema for Address Object Workflow
class AddressObjectState(TypedDict):
    """
    State schema for SCM address object creation workflow.
    
    This mirrors the structure from docs/examples/address_objects.py
    """
    # Input
    raw_input: str  # User's text input
    
    # Parsed fields
    name: str
    ip_netmask: str
    folder: str
    description: str
    
    # Validation
    validated: bool
    error_message: str
    
    # Final output
    api_ready_config: dict

print("✅ AddressObjectState schema defined")
print("\nThis state tracks the transformation:")
print("  Raw input → Parsed fields → Validated → API-ready config")

In [ ]:
# Step 2: Create the three workflow nodes

def parse_input(state: AddressObjectState) -> dict:
    """
    Node 1: Parse raw user input into structured fields.
    
    Expects input format: "create address <name> <ip/mask> in <folder>"
    Example: "create address web-server 10.1.1.100/32 in Texas"
    
    Args:
        state: Current workflow state
        
    Returns:
        Dictionary with parsed fields
    """
    print(f"📝 Parsing input: {state['raw_input']}")
    
    # Simple parsing logic (in production, use regex or proper parser)
    parts = state["raw_input"].split()
    
    if len(parts) < 6:
        return {
            "validated": False,
            "error_message": "Invalid input format. Expected: create address <name> <ip/mask> in <folder>"
        }
    
    name = parts[2]
    ip_netmask = parts[3]
    folder = parts[5]
    description = f"Address object for {name}"
    
    print(f"✅ Parsed: name={name}, ip_netmask={ip_netmask}, folder={folder}")
    
    return {
        "name": name,
        "ip_netmask": ip_netmask,
        "folder": folder,
        "description": description,
        "error_message": ""
    }


def validate_address(state: AddressObjectState) -> dict:
    """
    Node 2: Validate parsed address object fields.
    
    Checks:
    - IP address format is valid
    - Required fields are present
    - Folder name is valid
    
    Args:
        state: Current workflow state
        
    Returns:
        Dictionary with validation results
    """
    import re
    
    print(f"🔍 Validating address object: {state.get('name', 'unknown')}")
    
    # Check if parsing failed
    if state.get("error_message"):
        return {"validated": False}
    
    # Validate IP/netmask format
    ip_pattern = r'^(\d{1,3}\.){3}\d{1,3}/\d{1,2}$'
    
    if not re.match(ip_pattern, state["ip_netmask"]):
        print(f"❌ Invalid IP format: {state['ip_netmask']}")
        return {
            "validated": False,
            "error_message": f"Invalid IP format: {state['ip_netmask']}"
        }
    
    # Validate required fields are present
    if not state.get("name") or not state.get("folder"):
        print("❌ Missing required fields")
        return {
            "validated": False,
            "error_message": "Missing required fields (name or folder)"
        }
    
    print(f"✅ Validation passed for {state['name']}")
    return {
        "validated": True,
        "error_message": ""
    }


def format_for_api(state: AddressObjectState) -> dict:
    """
    Node 3: Transform validated state into SCM API format.
    
    Creates configuration matching the structure from docs/examples/address_objects.py.
    
    Production Pattern:
        See docs/examples/address_objects.py lines 9-15:
        netmask_config = {
            "name": "internal_network",
            "ip_netmask": "192.168.1.0/24",
            "description": "Internal network segment",
            "folder": "Texas",
            "tag": ["Python", "Automation"],
        }
    
    Args:
        state: Current workflow state
        
    Returns:
        Dictionary with API-ready configuration
    """
    print(f"🔧 Formatting configuration for SCM API")
    
    if not state["validated"]:
        print("⚠️  Skipping format - validation failed")
        return {"api_ready_config": {}}
    
    # Format according to docs/examples/address_objects.py lines 9-15
    # This config structure matches the pan-scm-sdk address object schema
    config = {
        "name": state["name"],
        "ip_netmask": state["ip_netmask"],
        "folder": state["folder"],
        "description": state["description"],
        "tag": ["Automation", "LangGraph"]
    }
    
    print(f"✅ API-ready configuration created:")
    pprint(config)
    
    return {"api_ready_config": config}


print("✅ Created three workflow nodes:")
print("   1. parse_input: Extracts fields from user input")
print("   2. validate_address: Validates IP format and required fields")
print("   3. format_for_api: Creates SCM API-ready configuration")
print("\n💡 The format_for_api node mirrors docs/examples/address_objects.py structure!")

In [ ]:
# Step 3: Build the graph with START → parse → validate → format → END

# Create StateGraph for address object workflow
address_graph = StateGraph(AddressObjectState)

# Add the three nodes
address_graph.add_node("parse", parse_input)
address_graph.add_node("validate", validate_address)
address_graph.add_node("format", format_for_api)

# Connect nodes in sequence
address_graph.add_edge(START, "parse")
address_graph.add_edge("parse", "validate")
address_graph.add_edge("validate", "format")
address_graph.add_edge("format", END)

# Compile to create runnable
address_app = address_graph.compile()

print("✅ SCM Address Object graph created and compiled")
print("\nGraph structure:")
print("  START → parse → validate → format → END")
print("\nVisualizing graph:")

In [ ]:
# Display the graph visualization
display(Image(address_app.get_graph().draw_mermaid_png()))

print("\n💡 This is the classic parse → validate → format pattern")
print("   used throughout configuration management workflows!")

In [ ]:
# Step 4: Test with valid input

print("="*60)
print("TEST 1: Valid Address Object Input")
print("="*60 + "\n")

valid_input: AddressObjectState = {
    "raw_input": "create address web-server 10.1.1.100/32 in Texas",
    "name": "",
    "ip_netmask": "",
    "folder": "",
    "description": "",
    "validated": False,
    "error_message": "",
    "api_ready_config": {}
}

print("Input:", valid_input["raw_input"])
print()

# Execute the workflow
result = address_app.invoke(valid_input)

print("\n" + "="*60)
print("FINAL STATE")
print("="*60)
pprint(result)

print("\n✅ Success! The workflow:")
print("   1. Parsed user input into structured fields")
print("   2. Validated IP address format")
print("   3. Created API-ready SCM configuration")
print("\n   This config is ready to send to the pan-scm-sdk!")

In [ ]:
# Step 5: Test with invalid input to see error handling

print("\n" + "="*60)
print("TEST 2: Invalid IP Format")
print("="*60 + "\n")

invalid_input: AddressObjectState = {
    "raw_input": "create address db-server 10.1.1.100 in Texas",  # Missing /mask!
    "name": "",
    "ip_netmask": "",
    "folder": "",
    "description": "",
    "validated": False,
    "error_message": "",
    "api_ready_config": {}
}

print("Input:", invalid_input["raw_input"])
print("Note: IP is missing the subnet mask!\n")

# Execute the workflow
result2 = address_app.invoke(invalid_input)

print("\n" + "="*60)
print("FINAL STATE")
print("="*60)
pprint(result2)

print("\n💡 The workflow caught the error:")
print(f"   validated: {result2['validated']}")
print(f"   error_message: {result2['error_message']}")
print("   api_ready_config: {} (empty - not created)")
print("\n   Validation prevented bad data from reaching the API!")

In [ ]:
print("\n" + "="*60)
print("Key Takeaways from SCM Address Object Workflow")
print("="*60)

print("""
This simple 3-node graph demonstrates fundamental LangGraph patterns:

1. **State Schema** (AddressObjectState)
   - Tracks the transformation from raw input to API-ready config
   - Uses TypedDict from Notebook 101 for type safety

2. **Sequential Nodes** (parse → validate → format)
   - Each node has a single responsibility
   - Node output updates state for next node
   - Classic configuration pipeline pattern

3. **Validation Gates**
   - Validation node checks data before proceeding
   - Invalid data stops the pipeline (validated=False)
   - Prevents bad configs from reaching the API

4. **API Integration Pattern**
   - Final node creates config matching pan-scm-sdk structure
   - From docs/examples/address_objects.py
   - Ready to send to SCM API

5. **Real-World Application**
   - User inputs text command
   - Graph transforms it into validated API request
   - Production-ready automation pattern

This is the foundation you'll use for more complex workflows:
- Security rule creation and validation
- Configuration change workflows
- Multi-step automation with conditional routing
- Error handling and rollback procedures

💡 In the next sections, you'll see how to add conditional routing
   and tool integration to make workflows even more powerful!
""")